In [10]:
import numpy as np 
import pandas as pd 
import datetime
from datetime import datetime
from tqdm import tqdm_notebook as tqdm

from multiprocessing import Pool
from functools import partial
import Feature_function as func

In [5]:
data = pd.read_csv('D:\\庫存健診開發\\data\\Stock\\TWSE_Stock_fundamental.csv', converters={'ts': str, 'StockName': str, 'StockNo': str})
data['ts'] = pd.to_datetime(data['ts'])
data

,StockName,StockNo,close,high,low,open,return,total,ts,vol,year,total_num,eliminate_period
0,台泥,1101,21.00,21.00,20.40,20.70,0.10,417236030.0,2004-12-20,20105028.0,2004,NaN,0
1,台泥,1101,20.20,20.20,19.70,19.70,0.50,100967115.0,2005-08-12,5037589.0,2005,NaN,0
2,台泥,1101,19.60,20.40,19.50,20.40,-0.80,420293307.0,2005-08-10,21295902.0,2005,NaN,0
3,台泥,1101,20.40,20.40,20.20,20.30,0.10,121245861.0,2005-08-24,5970108.0,2005,NaN,0
4,台泥,1101,20.40,20.55,20.20,20.50,-0.20,147516136.0,2005-08-09,7238224.0,2005,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2874551,世紀鋼,9958,11.30,11.80,11.10,11.80,-0.50,1950500.0,2008-07-08,170000.0,2008,NaN,0
2874552,世紀鋼,9958,19.70,19.80,19.40,19.50,0.30,14390595.0,2008-04-15,732635.0,2008,NaN,0
2874553,世紀鋼,9958,19.55,20.00,19.55,20.00,-0.15,18114525.0,2008-04-16,919500.0,2008,NaN,0
2874554,世紀鋼,9958,9.96,10.00,9.60,9.97,-0.01,2673890.0,2008-09-25,270000.0,2008,NaN,0


In [6]:
year_list = data["ts"].dt.year.unique().tolist()
month_list = data["ts"].dt.month.unique().tolist()
month_dict = {}
for y in year_list:
    month_dict[y] = {}
    for m in month_list:
        d = data[(data["ts"].dt.year == y) & (data["ts"].dt.month == m)]["ts"].dt.date.unique().tolist()
        month_dict[y][m] = {}
        for i, date in enumerate(d):
            month_dict[y][m][date] = i

In [7]:
df_list = [group[1] for group in data.groupby(data['StockNo'])]

output_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(partial(func.get_time_feature, month_dict=month_dict), df_list), total=len(df_list)), 1):
            output_list.append(x)
                
df = pd.concat(output_list, axis=0)
df

,StockName,StockNo,close,high,low,open,return,total,ts,vol,year,total_num,eliminate_period,Month,Weekday,Intramonth,IntramonthPeriod,WeekNum
0,亞泥,1102,27.90,27.95,27.55,27.75,-0.30,127083804.0,2018-03-02,4576443.0,2018,3.361447e+09,0,3,5,0.772727,4,0.00
1,亞泥,1102,30.70,31.05,30.10,30.75,-0.20,273865635.0,2018-04-25,8943756.0,2018,3.361447e+09,0,4,3,0.352941,2,0.80
2,亞泥,1102,30.95,31.15,30.45,30.50,0.55,320445273.0,2018-04-23,10355977.0,2018,3.361447e+09,0,4,1,0.941176,5,0.80
3,亞泥,1102,30.90,30.95,30.25,30.45,0.45,248606608.0,2018-01-17,8100450.0,2018,3.361447e+09,0,1,3,0.809524,5,0.50
4,亞泥,1102,30.75,30.95,30.60,30.95,-0.15,117607243.0,2018-01-18,3818598.0,2018,3.361447e+09,0,1,4,0.761905,4,0.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3861,新麗,9944,28.30,28.70,28.00,28.55,-0.25,9525634.0,2015-12-10,335003.0,2015,1.085732e+08,0,12,4,0.500000,3,0.25
3862,新麗,9944,28.05,28.50,27.80,28.50,-0.25,14793100.0,2015-12-11,528000.0,2015,1.085732e+08,0,12,5,0.409091,3,0.25
3863,新麗,9944,26.50,27.10,25.90,25.90,0.00,19859905.0,2015-12-15,743100.0,2015,1.085732e+08,0,12,2,0.772727,4,0.50
3864,新麗,9944,28.55,28.85,27.50,28.85,-0.30,7135885.0,2015-12-08,249003.0,2015,1.085732e+08,0,12,2,0.590909,3,0.25


In [8]:
df = func.categorical_transform(df)
df

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\sklearn\preprocessing\_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and

,StockName,StockNo,close,high,low,open,return,total,ts,vol,...,Tue,Wed,Thu,Fri,Sat,First,Second,Third,Fourth,Fifth
0,亞泥,1102,27.90,27.95,27.55,27.75,-0.30,127083804.0,2018-03-02,4576443.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,亞泥,1102,30.70,31.05,30.10,30.75,-0.20,273865635.0,2018-04-25,8943756.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,亞泥,1102,30.95,31.15,30.45,30.50,0.55,320445273.0,2018-04-23,10355977.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,亞泥,1102,30.90,30.95,30.25,30.45,0.45,248606608.0,2018-01-17,8100450.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,亞泥,1102,30.75,30.95,30.60,30.95,-0.15,117607243.0,2018-01-18,3818598.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2874551,新麗,9944,28.30,28.70,28.00,28.55,-0.25,9525634.0,2015-12-10,335003.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2874552,新麗,9944,28.05,28.50,27.80,28.50,-0.25,14793100.0,2015-12-11,528000.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2874553,新麗,9944,26.50,27.10,25.90,25.90,0.00,19859905.0,2015-12-15,743100.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2874554,新麗,9944,28.55,28.85,27.50,28.85,-0.30,7135885.0,2015-12-08,249003.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [11]:
df_list = [group[1] for group in df.groupby(df['StockNo'])]

output_list = []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, x in enumerate(tqdm(pool.imap_unordered(func.get_technical_indicators, df_list), total=len(df_list)), 1):
            output_list.append(x)
                
df = pd.concat(output_list, axis=0)
df

C:\Users\011553\AppData\Local\Continuum\anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.


,12ema,26ema,ATR,Apr,Aug,Dec,Feb,Fifth,First,Fourth,...,pvt,pvt_current,return,skew,total,total_num,ts,upper_band,vol,year
0,NaN,NaN,1.089254,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,NaN,NaN,NaN,NaN,1.895267e+08,NaN,2004-02-11,NaN,1.953132e+07,2004
1,NaN,NaN,1.083229,0.0,0.0,0.0,1.0,1.0,0.0,0.0,...,NaN,12555.716747,-0.5,NaN,2.468034e+08,NaN,2004-02-12,NaN,9.896328e+06,2004
2,NaN,NaN,1.076803,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,51064.067470,38508.350723,1.4,NaN,8.446286e+08,NaN,2004-02-13,NaN,2.937028e+07,2004
3,NaN,NaN,1.069958,0.0,0.0,0.0,1.0,0.0,0.0,1.0,...,85796.393050,47288.042327,0.4,NaN,1.132873e+09,NaN,2004-02-16,NaN,3.486248e+07,2004
4,NaN,NaN,1.062752,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,90856.887234,43568.844907,0.8,NaN,6.549583e+08,NaN,2004-02-17,NaN,3.180797e+07,2004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2848,75.056601,74.035946,2.053091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1074.106071,-6917.656338,-0.2,0.157269,2.371908e+08,209342910.0,2019-09-17,78.417724,4.322005e+06,2019
2849,75.211895,74.186322,1.998337,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-18256.480703,-11338.824364,0.4,-0.009699,4.648791e+08,209342910.0,2019-09-18,78.704141,3.066161e+06,2019
2850,75.325652,74.317062,1.936882,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-15054.547608,-3715.723244,-1.4,-0.197361,3.086962e+08,209342910.0,2019-09-19,78.937363,2.464138e+06,2019
2851,75.427366,74.440746,1.875160,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-2643.287531,1072.435713,-0.5,-0.399103,1.328813e+08,209342910.0,2019-09-20,79.127123,2.295856e+06,2019


In [12]:
df.columns

Index(['12ema', '26ema', 'ATR', 'Apr', 'Aug', 'Dec', 'Feb', 'Fifth', 'First',
       'Fourth', 'Fri', 'Intramonth', 'IntramonthPeriod', 'Jan', 'Jul', 'Jun',
       'MACD', 'Mar', 'May', 'Mon', 'Month', 'Nov', 'Oct', 'RSI_15', 'SO%d3',
       'SO%k5', 'Sat', 'Second', 'Sep', 'StockName', 'StockNo', 'TR', 'Third',
       'Thu', 'Tue', 'VWAP', 'Wed', 'WeekNum', 'Weekday', 'close',
       'eliminate_period', 'high', 'kurtosis', 'low', 'lower_band', 'ma21',
       'ma7', 'open', 'origin_close', 'origin_vol', 'pvt', 'pvt_current',
       'return', 'skew', 'total', 'total_num', 'ts', 'upper_band', 'vol',
       'year'],
      dtype='object')

In [13]:
df.to_csv('D:\\庫存健診開發\\data\\Stock\\TWSE_Stock_feature.csv', index=False)